In [ ]:
#import beautifulsoup
from bs4 import BeautifulSoup
import os
import pandas as pd
import sqlite3


In [ ]:
cd c:\\Users\\drake\\Documents\\WGU\\Capstone\\AnalyticsCapstone\\

In [ ]:
files = []
path = 'data/raw'

if not os.path.exists(path):
    print(f"Directory '{path}' does not exist. Please run load_data.py first to download the files.")

for filename in os.listdir(path):
    full_path = os.path.join(path, filename)
    if os.path.isfile(full_path):
        files.append(full_path)

print(f"Found {len(files)} files in {path}")
for file in files:
    print(f"  - {file}")

In [ ]:
#For each file, parse the html for the relevant stats and add it to the correct dataframe. Pitchers and batters have different stats, so they will be seperated.
batters_df = pd.DataFrame()
pitchers_df = pd.DataFrame()
num_files = len(files)
current_file = 0
for file in files:
    current_file += 1
    print(f"Processing file {current_file} of {num_files}: {file}")
    with open(file, "r", encoding="utf-8") as f:
        html_content = f.read()

    player_name = file.split(".html")[0].split("\\")[-1]
    
    # Parse the HTML
    soup = BeautifulSoup(html_content, "html.parser")

    # Find the player's salary
    meta = soup.find(id="meta")
    for p in meta.find_all("p"):
        if "Contract Status" in p.text:
            # Extract salary value using regex
            import re
            contract_text = p.text
            # Look for salary pattern like $20.5M or $15.2M
            salary_match = re.search(r'\$(\d+\.?\d*[MBK]?)', contract_text)
            if salary_match:
                salary = salary_match.group(1)  # Get just the value without the $
                print(f"Salary: ${salary}")
            else:
                print("No salary found in contract status")

        # Find the table wrapper for pitching
    div = soup.find(id="div_players_value_pitching")
    if div:
        row = div.find(id="players_value_pitching.2024")
        if row:
            stats_dict = {}
            stats_dict['fullName'] = player_name
            year = row.find("th", {"data-stat": "year_id"}).text.strip()
            stats_dict["year"] = year
            for cell in row.find_all("td"):
                stat_name = cell.get("data-stat")
                stat_value = cell.text.strip()
                stats_dict[stat_name] = stat_value
                stats_dict['salary'] = salary
            df = pd.DataFrame([stats_dict])
            pitchers_df = pd.concat([pitchers_df, df], ignore_index=True)
        else:
            print("2025 pitching row not found.")
    else:
        print("Pitching table not found.")
    
    # Now handle batting
    div = soup.find(id="div_players_standard_batting")
    if div:
        row = div.find(id="players_standard_batting.2024")
        if row:
            stats_dict = {}
            stats_dict['fullName'] = player_name
            year = row.find("th", {"data-stat": "year_id"}).text.strip()
            stats_dict["year"] = year
            for cell in row.find_all("td"):
                stat_name = cell.get("data-stat")
                stat_value = cell.text.strip()
                stats_dict[stat_name] = stat_value
                stats_dict['salary'] = salary
            df = pd.DataFrame([stats_dict])
            batters_df = pd.concat([batters_df, df], ignore_index=True)
        else:
            print("2025 batting row not found.")
    else:
        print("Batting table not found.")

# return the dataframes

In [ ]:
# Create database directory if it doesn't exist
db_dir = 'data/processed'
os.makedirs(db_dir, exist_ok=True)

# Create database connection
db_path = os.path.join(db_dir, 'baseball_stats.db')
conn = sqlite3.connect(db_path)

# Save pitchers dataframe to database
if not pitchers_df.empty:
    pitchers_df.to_sql('pitchers', conn, if_exists='replace', index=False)
    print(f"Saved {len(pitchers_df)} pitcher records to database")

# Save batters dataframe to database  
if not batters_df.empty:
    batters_df.to_sql('batters', conn, if_exists='replace', index=False)
    print(f"Saved {len(batters_df)} batter records to database")

# Close the connection
conn.close()
print(f"Database saved to: {db_path}")


In [ ]:
#The database now contains the parsed data from the web scraping, now we need to clean it.
conn = sqlite3.connect('data/processed/baseball_stats.db')
batters_df = pd.read_sql_query("SELECT * FROM batters", conn)

batters_df.head()

In [ ]:
batters_df.dtypes

In [ ]:
batters_df = batters_df.convert_dtypes()
# Manual overrides for known columns (example: year, salary)
if 'year' in batters_df.columns:
    batters_df['year'] = pd.to_numeric(batters_df['year'], errors='coerce').astype('Int64')
if 'age' in batters_df.columns:
    batters_df['age'] = pd.to_numeric(batters_df['age'], errors='coerce').astype('Int64')
if 'salary' in batters_df.columns:
    def parse_salary(val):
        if pd.isna(val):
            return None
        val = str(val).replace('$', '').replace(',', '').strip()
        if val.endswith('M'):
            return float(val[:-1]) * 1_000_000
        elif val.endswith('K'):
            return float(val[:-1]) * 1_000
        elif val.endswith('B'):
            return float(val[:-1]) * 1_000_000_000
        else:
            try:
                return float(val)
            except Exception:
                return None
    batters_df['salary'] = batters_df['salary'].map(parse_salary).astype('float')
# Numeric columns to float
float_cols = [
    'b_war', 'b_batting_avg', 'b_onbase_perc', 'b_slugging_perc', 'b_onbase_plus_slugging',
    'b_roba'
]
for col in float_cols:
    if col in batters_df.columns:
        batters_df[col] = pd.to_numeric(batters_df[col].astype(str).str.replace('^\.', '0.', regex=True), errors='coerce').astype('float')
# Integer columns
int_cols = [
    'b_games', 'b_pa', 'b_ab', 'b_r', 'b_h', 'b_doubles', 'b_triples', 'b_hr', 'b_rbi',
    'b_sb', 'b_cs', 'b_bb', 'b_so', 'b_onbase_plus_slugging_plus', 'b_rbat_plus', 'b_tb',
    'b_gidp', 'b_hbp', 'b_sh', 'b_sf', 'b_ibb', 'awards', 'b_rbat_plus'
]
for col in int_cols:
    if col in batters_df.columns:
        batters_df[col] = pd.to_numeric(batters_df[col], errors='coerce').astype('Int64')
# Categorical/text columns: fullName, team_name_abbr, comp_name_abbr, pos, awards
for col in ['fullName', 'team_name_abbr', 'comp_name_abbr', 'pos', 'awards']:
    if col in batters_df.columns:
        batters_df[col] = batters_df[col].astype('string')

batters_df.info()

In [ ]:
# For our analysis, we do not need the age or awards columns.
batters_df = batters_df.drop(columns=['age', 'awards'])

In [ ]:
# Now for the pitchers.
conn = sqlite3.connect('data/processed/baseball_stats.db')
pitchers_df = pd.read_sql_query("SELECT * FROM pitchers", conn)
pitchers_df.iloc[0].to_dict()

In [ ]:
# We will need to update our cleaning method to include the pitchers.
# Use pandas' convert_dtypes for best-guess
pitchers_df = pitchers_df.convert_dtypes()
# Manual overrides for known columns (example: year, salary)
if 'year' in pitchers_df.columns:
    pitchers_df['year'] = pd.to_numeric(pitchers_df['year'], errors='coerce').astype('Int64')
if 'age' in pitchers_df.columns:
    pitchers_df['age'] = pd.to_numeric(pitchers_df['age'], errors='coerce').astype('Int64')
if 'salary' in pitchers_df.columns:
    def parse_salary(val):
        if pd.isna(val):
            return None
        val = str(val).replace('$', '').replace(',', '').strip()
        if val.endswith('M'):
            return float(val[:-1]) * 1_000_000
        elif val.endswith('K'):
            return float(val[:-1]) * 1_000
        elif val.endswith('B'):
            return float(val[:-1]) * 1_000_000_000
        else:
            try:
                return float(val)
            except Exception:
                return None
    pitchers_df['salary'] = pitchers_df['salary'].map(parse_salary).astype('float')
# Batting float columns
float_cols = [
    'b_war', 'b_batting_avg', 'b_onbase_perc', 'b_slugging_perc', 'b_onbase_plus_slugging',
    'b_roba'
]
# Pitching float columns
float_cols += [
    'p_ip', 'p_ra9', 'p_ra9_opp', 'p_ra9_def', 'p_ra9_role', 'p_ra9_extras',
    'p_ra9_avg_pitcher', 'p_waa', 'p_leverage_index_avg_rp', 'p_waa_adj', 'p_war',
    'p_waa_win_perc', 'p_waa_win_perc_162'
]
for col in float_cols:
    if col in pitchers_df.columns:
        pitchers_df[col] = pd.to_numeric(pitchers_df[col].astype(str).str.replace('^\.', '0.', regex=True), errors='coerce').astype('float')
# Batting integer columns
int_cols = [
    'b_games', 'b_pa', 'b_ab', 'b_r', 'b_h', 'b_doubles', 'b_triples', 'b_hr', 'b_rbi',
    'b_sb', 'b_cs', 'b_bb', 'b_so', 'b_onbase_plus_slugging_plus', 'b_rbat_plus', 'b_tb',
    'b_gidp', 'b_hbp', 'b_sh', 'b_sf', 'b_ibb', 'awards', 'b_rbat_plus'
]
# Pitching integer columns
int_cols += [
    'p_g', 'p_gs', 'p_r', 'p_ppf_custom', 'p_raa', 'p_rar'
]
for col in int_cols:
    if col in pitchers_df.columns:
        pitchers_df[col] = pd.to_numeric(pitchers_df[col], errors='coerce').astype('Int64')
# Categorical/text columns: fullName, team_name_abbr, comp_name_abbr, pos, awards
for col in ['fullName', 'team_name_abbr', 'comp_name_abbr', 'pos', 'awards']:
    if col in pitchers_df.columns:
        pitchers_df[col] = pitchers_df[col].astype('string')

pitchers_df.info()

In [ ]:
#As before, drop the age and awards columns.
pitchers_df = pitchers_df.drop(columns=['awards'])

In [ ]:
#the p_leverage_index_avg_rp column contains many missing values, so we will drop it.
pitchers_df = pitchers_df.drop(columns=['p_leverage_index_avg_rp'])

In [ ]:
#Lets take a look at the cleaned data.
batters_df.head()

In [ ]:
pitchers_df.head()